# Exercise 3 from the intro to python gis course
https://github.com/AutoGIS-2017/Exercise-3

In [1]:
import geopandas as gpd
from geopandas.tools import geocode
import pandas as pd
from shapely.geometry import Point, LineString, Polygon
import fiona

## Problem 1: Geocode shopping centers (5 points)

In the first problem the aim is to find out the addresses of shopping centers and geocoding them as a single Shapefile called `shopping_centers.shp`.

**Steps**

- From the internet find out the addresses for following shopping centers and write the addresses into a text file called `shopping_centers.txt`:

 - Itis
 - Forum
 - Iso-omena
 - Sello
 - Jumbo
 - REDI (i.e. use the metro station of Kalasatama)

Use same kind of formatting for the text file as in the [lesson materials](https://automating-gis-processes.github.io/2017/lessons/L3/geocoding.html#geocoding-in-geopandas), thus use semicolon `;` as a separator and add a unique integer number as `id` (doesn't matter what) for each center. 

- Geocode the addresses in Geopandas in a similar manner as was done in the [lesson materials](https://automating-gis-processes.github.io/2016/Lesson3-geocoding.html#geocoding-in-geopandas)

- Reproject the geometries into a EPSG projection 3879 similarly as in [lesson materials](https://automating-gis-processes.github.io/2016/Lesson3-projections.html#projections-converting-from-projection-to-another)
  
   - **Notice**: you need to pass the coordinate information as a proj 4 dictionary in a similar manner as in the lesson materials (see the second last bullet point in the [lesson materials](https://automating-gis-processes.github.io/2016/Lesson3-projections.html#projections-converting-from-projection-to-another)

- Make a Table join to retrieve the `id` column from original shopping centers DataFrame similarly as in [lesson materials](https://automating-gis-processes.github.io/2016/Lesson3-table-join.html)

- Save the GeoDataFrame as a Shapefile called `shopping_centers.shp`

In [2]:
shoppingCenterAddresses = pd.read_csv('./shopping_centers.csv', delimiter=';')
shoppingCenters = geocode(shoppingCenterAddresses['addr'], provider='nominatim')

In [3]:
shoppingCenters['geometry'] = shoppingCenters['geometry'].to_crs(epsg=3879)

In [4]:
shoppingCenters.join(shoppingCenterAddresses)
shoppingCenters.to_file("shopping_centers.shp")

## Problem 2: Create buffers around shopping centers (5 points)

Let's continue with our case study and calculate a 5 km `buffer` around the points. 

**Steps**

- Create a new column called `buffer` to your shopping-centers GeoDataFrame (or whatever you call it)

- Iterate over the rows in your GeoDataFrame and update the `buffer` column with a 5 km buffer Polygon.
  
  - Use Shapely's [buffer](http://toblerity.org/shapely/manual.html#object.buffer) function to create it (see the link for details how to use it)
  - You only need to use the `distance` -parameter, don't care about the other parameters.
  
- Replace the values in `geometry` column with the values of `buffer` column

In [5]:
shoppingCenterPoints = shoppingCenters['geometry'].to_list()
shoppingCenterPointBuffers = [point.buffer(5000) for point in shoppingCenterPoints]
shoppingCenters['geometry'] = shoppingCenterPointBuffers

## Problem 3: How many people live within 5 km from shopping centers? (5 points)

Last step in our analysis is to make a spatial join between our point-buffer layer and the same population grid that was [used in the lesson materials](https://automating-gis-processes.github.io/2016/Lesson3-spatial-join.html#download-and-clean-the-data).

**Steps**

- Read and prepare the population grid into a GeoDataFrame similarly as in [the lesson materials](https://automating-gis-processes.github.io/2016/Lesson3-spatial-join.html#download-and-clean-the-data)

- Make a spatial join between your buffered point layer and population grid layer

  - Note: Join the information now from buffer layer **into the population grid layer**

- Group the joined layer by shopping center index

- Calculate the sum of population living within 5 km for each shopping center.
  
  - Write the answers down here into the [Answers](#answers) section

In [6]:
pop = gpd.read_file('./Vaestotietoruudukko_2015/Vaestotietoruudukko_2015.shp')
pop.rename(columns={'ASUKKAITA': 'population'}, inplace=True)
pop = pop[['geometry', 'population']]

In [7]:
pop.crs == shoppingCenters.crs
pop['geometry'] = pop['geometry'].to_crs(epsg=3879)

In [8]:
shoppingPop = gpd.sjoin(shoppingCenters, pop, how="inner", op='intersects')
shoppingPop.drop(['index_right'], axis=1, inplace=True)
shoppingPop.head()

/anaconda3/envs/pythonGIS/lib/python3.7/site-packages/geopandas/tools/sjoin.py:56: UserWarning: CRS of frames being joined does not match!({'init': 'epsg:4326', 'no_defs': True} != {'init': 'epsg:3879'})
  '(%s != %s)' % (left_df.crs, right_df.crs))


,geometry,address,population
0,"POLYGON ((25509598.60162312 6677662.108634275,...","Kauppakeskus Itis, 1-7, Itäkatu, Itäkeskus, Va...",579
0,"POLYGON ((25509598.60162312 6677662.108634275,...","Kauppakeskus Itis, 1-7, Itäkatu, Itäkeskus, Va...",370
0,"POLYGON ((25509598.60162312 6677662.108634275,...","Kauppakeskus Itis, 1-7, Itäkatu, Itäkeskus, Va...",674
0,"POLYGON ((25509598.60162312 6677662.108634275,...","Kauppakeskus Itis, 1-7, Itäkatu, Itäkeskus, Va...",136
0,"POLYGON ((25509598.60162312 6677662.108634275,...","Kauppakeskus Itis, 1-7, Itäkatu, Itäkeskus, Va...",276


In [9]:
shoppingPop.groupby(['address']).sum()

,population
address,
"3-9, Leppävaarankatu, Etelä-Leppävaara, Leppävaara, Suur-Leppävaara, Espoo, Helsingin seutukunta, Uusimaa, Etelä-Suomi, Manner-Suomi, 02600, Suomi",196257
"Iso Omena, 11, Piispansilta, Espoo, Helsingin seutukunta, Uusimaa, Etelä-Suomi, Manner-Suomi, 02230, Suomi",116855
"Kauppakeskus Itis, 1-7, Itäkatu, Itäkeskus, Vartiokylä, Helsinki, Helsingin seutukunta, Uusimaa, Etelä-Suomi, Manner-Suomi, 00930, Suomi",183140
"Salaattiasema, 14-20, Mannerheimintie, Keskusta, Kluuvi, Eteläinen suurpiiri, Helsinki, Helsingin seutukunta, Uusimaa, Etelä-Suomi, Manner-Suomi, 00100, Suomi",224565
"Stockmann, 3, Vantaanportinkatu, Pakkala, Aviapoliksen suuralue, Vantaa, Helsingin seutukunta, Uusimaa, Etelä-Suomi, Manner-Suomi, 01510, Suomi",108346
"Sörnäinen, Etelä-Suomi, Manner-Suomi, 00580, Suomi",267338
